# Regression to learn Connect Rate

In [51]:
# Read the Data from S3

import boto3
import pandas as pd
from sagemaker import get_execution_role
role = get_execution_role()
bucket='apjprintsupply'
data_location = 's3://{}/APJ_Supplies/SampleData_Deconstruct_transformed.csv'.format(bucket)
deconstruct=pd.read_csv(data_location)


In [52]:
# Look at  rows to ensure data is correct
deconstruct

,Country,SKUID,Printer_Name,RS_Category,Prod_Fam_Name,at_month,Total_Shipments,Lag_IB,FY_QTR,Year,...,COL_CR,ARU_Black,ARU_Color,Black_SO_unit,Color_SO_unit,Black_SO_Rev,Color_SO_Rev,Total_REVENUE,Total_UNIT,TOTAL_ARU
0,CHINA,14,Printer A,Printer A00 Series,NaN,201407,70,68.63661,2014Q4,2014,...,18.436634,21.168732,17.722689,656.804849,1265.428125,13903.725640,22426.788850,36330.514490,1922.232974,18.900162
1,INDONESIA,15,Printer B,Printer B00 Series,NaN,201410,181,175.99262,2015Q1,2015,...,7.404018,19.156825,17.140243,615.784763,1303.052611,11796.480920,22334.638020,34131.118940,1918.837374,17.787395
2,INDONESIA,16,Printer C,Printer C00 Series,NaN,201501,193,183.18033,2015Q2,2015,...,13.092670,16.854973,16.236971,1165.799737,2398.319645,19649.523150,38941.446350,58590.969500,3564.119382,16.439115
3,MALAYSIA,17,Printer D,Printer D00 Series,NaN,201504,208,193.08634,2015Q3,2015,...,1.427464,15.079788,16.639820,605.468511,275.623725,9130.336647,4586.329082,13716.665730,881.092237,15.567798
4,MALAYSIA,18,Printer E,Printer E00 Series,NaN,201507,258,237.19302,2015Q4,2015,...,0.829286,20.878317,16.026310,94.546833,196.700883,1973.978789,3152.389237,5126.368026,291.247717,17.601402
5,CHINA,19,Printer C,Printer A00 Series,NaN,201510,265,237.69870,2016Q1,2016,...,0.444518,18.977260,16.570932,60.081960,105.661378,1140.190969,1750.907477,2891.098445,165.743338,17.443226
6,INDONESIA,20,Printer D,Printer B00 Series,NaN,201601,301,266.74373,2016Q2,2016,...,0.420234,20.563380,16.984272,65.904919,112.094691,1355.227885,1903.846732,3259.074617,177.999610,18.309448
7,MALAYSIA,21,Printer E,Printer C00 Series,NaN,201604,301,259.19645,2016Q3,2016,...,3.323723,20.706970,16.538694,833.544753,861.497293,17260.186490,14248.039690,31508.226180,1695.042047,18.588463
8,INDONESIA,22,Printer C,Printer D00 Series,NaN,201607,301,251.26832,2016Q4,2016,...,4.579643,22.215230,17.222154,599.227579,1150.719128,13311.978660,19817.861720,33129.840370,1749.946708,18.931914
9,SINGAPORE,23,Printer D,Printer E00 Series,NaN,201610,301,242.71759,2017Q1,2017,...,0.975296,17.941353,16.998327,180.359250,236.721515,3235.888876,4023.869788,7259.758664,417.080765,17.406122


In [57]:
#Drop Rows with invalid data
#deconstructSliced= deconstruct.drop(deconstruct.index[[12]])
deconstructSliced= deconstruct

In [65]:
# Split Data into train and test blocks
from sklearn.model_selection import train_test_split
train, test = train_test_split(deconstructSliced, test_size=0.2)

In [70]:
# Identify variable to be predicted and Features to be used
y = deconstructSliced.TOTAL_ARU
features = ['SKUID','at_month', 'Total_Shipments']
X = deconstructSliced[features]
X.describe()

,SKUID,at_month,Total_Shipments
count,12.000000,12.000000,12.000000
mean,19.500000,201555.500000,248.416667
std,3.605551,98.412398,72.637904
min,14.000000,201407.000000,70.000000
25%,16.750000,201503.250000,204.250000
50%,19.500000,201555.500000,283.000000
75%,22.250000,201607.750000,301.000000
max,25.000000,201704.000000,301.000000


In [71]:
#Build Model
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
deconstruct_model = DecisionTreeRegressor(random_state=1)

# Fit model
deconstruct_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')

In [72]:
print("Making predictions for the following 5 SKUs:")
print(X.head())
print("The predictions are")
print(deconstruct_model.predict(X.head()))

Making predictions for the following 5 SKUs:
   SKUID  at_month  Total_Shipments
0     14    201407               70
1     15    201410              181
2     16    201501              193
3     17    201504              208
4     18    201507              258
The predictions are
[18.90016193 17.78739533 16.43911531 15.56779773 17.60140159]
